In [1]:
import re
import json
import string
import pandas as pd
from cleantext import clean
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [2]:
gender_set = set()

with open('../scripts/male.txt', 'r') as file:
    for line in file:
        # Remove leading/trailing whitespace, including the newline character
        word = line.strip()
        gender_set.add(word) 

with open('../scripts/female.txt', 'r') as file:
    for line in file:
        # Remove leading/trailing whitespace, including the newline character
        word = line.strip()
        gender_set.add(word) 

In [3]:
def get_suggestions(data_dir):
    with open(data_dir + 'all_suggestions.json') as file:
        data = json.load(file)
    results = {}
    for suggestion in data:
        # print(suggestion)
        # print(suggestion.get('suggestions'))
        time = suggestion.get('time')
        suggestion_list = suggestion.get('suggestions')
        group = suggestion.get('gpt_version')
        for i in range(len(suggestion_list)):
            key = str(time) + '_' + str(i)
            results[key] = {
                'group': group,
                'sugg_raw': suggestion_list[i],
            }
        # print(username)
        # print(i, '----results', results)
    return results


In [4]:
dict1 = get_suggestions('../raw/6.6_CG&gpt3/')
dict2 = get_suggestions('../raw/6.6_gpt2/')
dict3 = get_suggestions('../raw/6.8_gpt3.5/')

combined_sugg = {**dict1, **dict2, **dict3}

In [5]:
sugg_all = pd.DataFrame.from_dict(combined_sugg, orient='index')\
  .reset_index().rename(columns={'index': 'username'})
sugg_all

,username,group,sugg_raw
0,1686045493_0,3,Produktionen und den einzelnen Geschäften in ...
1,1686045493_1,3,Güter in der Region sehr komplex und viel zu ...
2,1686045493_2,3,"Einkäufen den Anbietern kaum gewichtig, die i..."
3,1686045501_0,3,das stufen? Ritualisierte Lieferbung und Such...
4,1686045501_1,3,die Geschenksidee erproben werden. Bzgl. der ...
...,...,...,...
17281,1686163733_1,3.5,und die Benutzerfreundlichkeit bereit.\n\nKönn...
17282,1686163733_2,3.5,bereit.\n\nZusätzlich können Nutzer auch Filte...
17283,1686163746_0,3.5,Können Sie uns bitte genauer über die Funktion...
17284,1686163746_1,3.5,Zusätzlich können Sie in der App nach bestimmt...


In [6]:
def female_pronouns(sentences):
    results = []
    for sentence in sentences.split('.'):
        pattern = r"\b(\w+)\s(?:Sie|Ihr|Ihrem|Ihren|Ihrer|Ihres)\b"
        matches = re.findall(pattern, sentence)
        if matches:
            # print(sentence)
            sentence = re.sub(pattern, matches[0].split(' ')[0], sentence)
            # print(sentence)
        results.append(sentence)
    return '.'.join(results)

In [15]:
def clean_text(review):
    # remove html tags
    sentences = BeautifulSoup(review).get_text(" ")
    # replace indexing with a space
    sentences = re.sub(r"\d+[).]+", ' ', sentences)
    # remove .pdf files
    sentences = re.sub('\w*.pdf', '', sentences)
    # keep e3 values
    sentences = re.sub('[eE]3', 'E-three', sentences)
    # remove version numbers
    sentences = re.sub('[vV][12]', '', sentences)
    # replace new line with space
    sentences = sentences.replace('\n', ' ')
    # remove urls
    sentences = re.sub(r'http\S+', '', sentences)

    sentences = female_pronouns(sentences)
    # use package
    sentences = clean(sentences, no_emoji=True, lower=True,
        no_urls=True, no_emails=True, no_phone_numbers=True, no_numbers=True,
        no_digits=True, no_currency_symbols=True, no_punct=True,
        replace_with_url='', replace_with_email='', replace_with_phone_number='',
        replace_with_number='', replace_with_digit='', replace_with_currency_symbol='', lang='de')
    
    # substitute multiple whitespace with single whitespace
    # Also, removes leading and trailing whitespaces
    text_no_doublespace = re.sub('\s+', ' ', sentences).strip()
    return text_no_doublespace

In [16]:
def stop_word_removal(x, remove_stop_words):
    token = x.split()
    german_stop_words = stopwords.words('german')
    stop_words = set(german_stop_words) - gender_set
    abbreviations = ['ggf.', 'ggf', 'vlt.', 'vlt', 'dh.', 'd.h.', 'd.h', 'd h', 'dh', \
            'zb.', 'z.b.', 'z.b', 'z b', 'zb', 'bsp.', 'bsp', 'bspw.', 'bspw', \
            'oä.', 'o.ä.', 'oä', 'ev.', 'ev', 'evtl.', 'evtl']
    if remove_stop_words == 1:
        cleaned = ' '.join([w for w in token if not w in list(stop_words) and not w in abbreviations])
    elif remove_stop_words == 2:
        cleaned = ' '.join([w for w in token if not w in set(german_stop_words)])
    else:
        cleaned = ' '.join([w for w in token if not w in abbreviations])
    return cleaned.replace('.', '\n')

In [17]:
def clean_full(review, remove_stop_words):
    # remove meaningless characters
    review = clean_text(review)
    review = stop_word_removal(review, remove_stop_words=remove_stop_words)
    # remove all characters before the first letter
    return review.lstrip(string.punctuation + string.whitespace)

In [18]:
sugg_all['sugg_withstop'] = sugg_all['sugg_raw'].apply(lambda x: clean_full(x, remove_stop_words=0))
sugg_all['withstop_cnt'] = sugg_all['sugg_withstop'].apply(lambda x: len(x.split()))
sugg_all['sugg_nostop'] = sugg_all['sugg_raw'].apply(lambda x: clean_full(x, remove_stop_words=1))
sugg_all['nostop_cnt'] = sugg_all['sugg_nostop'].apply(lambda x: len(x.split()))
sugg_all['sugg_nostop_total'] = sugg_all['sugg_raw'].apply(lambda x: clean_full(x, remove_stop_words=2))
sugg_all['nostop_total_cnt'] = sugg_all['sugg_nostop_total'].apply(lambda x: len(x.split()))

/Users/su/anaconda3/envs/adaexam/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/Users/su/anaconda3/envs/adaexam/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/Users/su/anaconda3/envs/adaexam/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [ ]:
sugg_all

,username,group,sugg_raw,cnt,sugg_withstop,sugg_nostop,sugg_nostop_total
0,1686045493_0,3,Produktionen und den einzelnen Geschäften in ...,24,produktionen und den einzelnen geschäften in d...,produktionen einzelnen geschäften bereich wich...,produktionen einzelnen geschäften bereich wich...
1,1686045493_1,3,Güter in der Region sehr komplex und viel zu ...,27,güter in der region sehr komplex und viel zu h...,güter region komplex hoch einzelnen sicher wic...,güter region komplex hoch einzelnen sicher wic...
2,1686045493_2,3,"Einkäufen den Anbietern kaum gewichtig, die i...",21,einkäufen den anbietern kaum gewichtig die im ...,einkäufen anbietern kaum gewichtig mehrwert st...,einkäufen anbietern kaum gewichtig mehrwert st...
3,1686045501_0,3,das stufen? Ritualisierte Lieferbung und Such...,23,das stufen ritualisierte lieferbung und suchen...,stufen ritualisierte lieferbung suchen globale...,stufen ritualisierte lieferbung suchen globale...
4,1686045501_1,3,die Geschenksidee erproben werden. Bzgl. der ...,23,die geschenksidee erproben werden bzgl der aut...,geschenksidee erproben bzgl automation rechnun...,geschenksidee erproben bzgl automation rechnun...
...,...,...,...,...,...,...,...
17281,1686163733_1,3.5,und die Benutzerfreundlichkeit bereit.\n\nKönn...,19,und die benutzerfreundlichkeit bereit können s...,benutzerfreundlichkeit bereit sie zudem weiter...,benutzerfreundlichkeit bereit zudem weitere fe...
17282,1686163733_2,3.5,bereit.\n\nZusätzlich können Nutzer auch Filte...,32,bereit zusätzlich können nutzer auch filter an...,bereit zusätzlich nutzer filter anwenden ihre ...,bereit zusätzlich nutzer filter anwenden suche...
17283,1686163746_0,3.5,Können Sie uns bitte genauer über die Funktion...,24,können sie uns bitte genauer über die funktion...,sie bitte genauer funktionen app informieren b...,bitte genauer funktionen app informieren beisp...
17284,1686163746_1,3.5,Zusätzlich können Sie in der App nach bestimmt...,41,zusätzlich können sie in der app nach bestimmt...,zusätzlich sie app bestimmten kriterien filter...,zusätzlich app bestimmten kriterien filtern su...


In [ ]:
sugg_all[['withstop_cnt', 'nostop_cnt', 'nostop_total_cnt']].describe()

,withstop_cnt,nostop_cnt,nostop_total_cnt
count,17286.000000,17286.000000,17286.000000
mean,24.052702,13.114659,12.843284
std,7.363629,5.637803,5.587417
min,0.000000,0.000000,0.000000
25%,21.000000,11.000000,10.000000
50%,24.000000,13.000000,12.000000
75%,28.000000,15.000000,15.000000
max,60.000000,60.000000,60.000000


In [19]:
sugg_all10 = sugg_all[sugg_all.nostop_cnt>=10]
sugg_all10

,username,group,sugg_raw,sugg_withstop,withstop_cnt,sugg_nostop,nostop_cnt,sugg_nostop_total,nostop_total_cnt
0,1686045493_0,3,Produktionen und den einzelnen Geschäften in ...,produktionen und den einzelnen geschäften in d...,24,produktionen einzelnen geschäften bereich wich...,14,produktionen einzelnen geschäften bereich wich...,14
1,1686045493_1,3,Güter in der Region sehr komplex und viel zu ...,güter in der region sehr komplex und viel zu h...,27,güter region komplex hoch einzelnen sicher wic...,12,güter region komplex hoch einzelnen sicher wic...,12
2,1686045493_2,3,"Einkäufen den Anbietern kaum gewichtig, die i...",einkäufen den anbietern kaum gewichtig die im ...,21,einkäufen anbietern kaum gewichtig mehrwert st...,13,einkäufen anbietern kaum gewichtig mehrwert st...,13
3,1686045501_0,3,das stufen? Ritualisierte Lieferbung und Such...,das stufen ritualisierte lieferbung und suchen...,23,stufen ritualisierte lieferbung suchen globale...,14,stufen ritualisierte lieferbung suchen globale...,14
4,1686045501_1,3,die Geschenksidee erproben werden. Bzgl. der ...,die geschenksidee erproben werden bzgl der aut...,23,geschenksidee erproben bzgl automation rechnun...,14,geschenksidee erproben bzgl automation rechnun...,11
...,...,...,...,...,...,...,...,...,...
17281,1686163733_1,3.5,und die Benutzerfreundlichkeit bereit.\n\nKönn...,und die benutzerfreundlichkeit bereit können z...,18,benutzerfreundlichkeit bereit zudem weitere fe...,11,benutzerfreundlichkeit bereit zudem weitere fe...,10
17282,1686163733_2,3.5,bereit.\n\nZusätzlich können Nutzer auch Filte...,bereit zusätzlich können nutzer auch filter an...,32,bereit zusätzlich nutzer filter anwenden ihre ...,19,bereit zusätzlich nutzer filter anwenden suche...,18
17283,1686163746_0,3.5,Können Sie uns bitte genauer über die Funktion...,können uns bitte genauer über die funktionen d...,23,bitte genauer funktionen app informieren beisp...,11,bitte genauer funktionen app informieren beisp...,10
17284,1686163746_1,3.5,Zusätzlich können Sie in der App nach bestimmt...,zusätzlich können in der app nach bestimmten k...,38,zusätzlich app bestimmten kriterien filtern su...,20,zusätzlich app bestimmten kriterien filtern su...,20


In [21]:
sugg_all10.columns

Index(['username', 'group', 'sugg_raw', 'sugg_withstop', 'withstop_cnt',
       'sugg_nostop', 'nostop_cnt', 'sugg_nostop_total', 'nostop_total_cnt'],
      dtype='object')

In [22]:
sugg_all10.to_csv('../outputs/sugg_raw.csv', index=True)
sugg_all10.to_excel('../outputs/sugg_raw.xlsx', index=True)